In [4]:
import numpy as np

bbox_annos = np.load("./files/Google Drive/bbox_annos.npy", allow_pickle=True).item()
bbox_annos_sample = {key: bbox_annos[key] for key in list(bbox_annos.keys())[:10]}
bbox_annos_sample

{'fork_000000537944.jpg': [56, 229, 33, 49],
 'knife_000000233539.jpg': [372, 193, 114, 92],
 'tv_000000192651.jpg': [51, 138, 60, 54],
 'laptop_000000335308.jpg': [16, 210, 52, 39],
 'sink_000000164725.jpg': [158, 219, 159, 94],
 'fork_000000061672.jpg': [383, 101, 51, 141],
 'laptop_000000204979.jpg': [48, 98, 130, 101],
 'mouse_000000539056.jpg': [386, 206, 41, 44],
 'fork_000000323370.jpg': [399, 55, 65, 184],
 'tv_000000203160.jpg': [98, 57, 71, 71]}

In [ ]:
import numpy as np

bbox_annos = np.load("./files/Google Drive/bbox_annos.npy", allow_pickle=True).item()
bbox_annos_sample = {key: bbox_annos[key] for key in list(bbox_annos.keys())[:10]}
bbox_annos_sample

In [7]:
import json
import random

# Replace 'data.json' with the path to your original JSON file
with open('files/DCBs_JSONs/dataset_test/SAIL_fixations_TP_train.json', 'r') as file:
    data = json.load(file)

# Shuffle the data to ensure random splitting
random.shuffle(data)

# Calculate the split index for 80/20 split
split_index = int(len(data) * 0.8)

# Split the data into training and validation sets
train_data = data[:split_index]
valid_data = data[split_index:]

# Write the training data to 'human_scanpath_train_split.json'
with open('files/human_scanpath_train_split.json', 'w') as train_file:
    json.dump(train_data, train_file, indent=4)

# Write the validation data to 'human_scanpath_valid_split.json'
with open('files/human_scanpath_valid_split.json', 'w') as valid_file:
    json.dump(valid_data, valid_file, indent=4)

print("Data has been split and written to the respective files.")

Data has been split and written to the respective files.


In [22]:
import json
import os

# Specify the path to your JSON file
json_file_path = 'fixation_data.json'

# Check if the file exists
if not os.path.exists(json_file_path):
    print(f"File not found: {json_file_path}")
    exit(1)

# Load the JSON data
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Initialize counters and lists to store entries exceeding thresholds
entries_exceeding_threshold = []
total_entries = len(data)
entries_with_exceeding_values = 0

# Thresholds
x_threshold = 512
y_threshold = 320

# Iterate over each entry in the data
for idx, entry in enumerate(data):
    X_values = entry.get('X', [])
    Y_values = entry.get('Y', [])
    entry_name = entry.get('name', f'Entry {idx}')

    # Flags to check if any value exceeds the thresholds
    x_exceeds = any(x > x_threshold for x in X_values)
    y_exceeds = any(y > y_threshold for y in Y_values)

    # If any value exceeds the thresholds, store the entry information
    if x_exceeds or y_exceeds:
        entries_with_exceeding_values += 1
        entries_exceeding_threshold.append({
            'entry_index': idx,
            'name': entry_name,
            'X_exceeds': [x for x in X_values if x > x_threshold],
            'Y_exceeds': [y for y in Y_values if y > y_threshold]
        })

# Reporting the results
print(f"Total entries checked: {total_entries}")
print(f"Entries with values exceeding thresholds: {entries_with_exceeding_values}\n")

if entries_exceeding_threshold:
    print("Entries exceeding thresholds:")
    for item in entries_exceeding_threshold:
        print(f"\nEntry Index: {item['entry_index']}")
        print(f"Name: {item['name']}")
        if item['X_exceeds']:
            print(f"X values exceeding {x_threshold}: {item['X_exceeds']}")
        if item['Y_exceeds']:
            print(f"Y values exceeding {y_threshold}: {item['Y_exceeds']}")
else:
    print("No entries have values exceeding the specified thresholds.")

Total entries checked: 198
Entries with values exceeding thresholds: 0

No entries have values exceeding the specified thresholds.


In [16]:
import json
import random
import os

# Specify the path to your JSON file
json_file_path = 'DCBs_JSONs/dataset_test/SAIL_fixations_TA_train.json'

# Output file paths
train_file_path = 'human_scanpath_train_split.json'
valid_file_path = 'human_scanpath_valid_split.json'

# Check if the input file exists
if not os.path.exists(json_file_path):
    print(f"File not found: {json_file_path}")
    exit(1)

# Load the JSON data
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Thresholds
x_threshold = 512
y_threshold = 320

# Filter entries that have no X[i] > 512 and no Y[i] > 320
filtered_data = []
for idx, entry in enumerate(data):
    X_values = entry.get('X', [])
    Y_values = entry.get('Y', [])

    # Check if any X[i] > 512 or Y[i] > 320
    x_exceeds = any(x > x_threshold for x in X_values)
    y_exceeds = any(y > y_threshold for y in Y_values)

    if not x_exceeds and not y_exceeds:
        filtered_data.append(entry)

# Shuffle the filtered data
random.shuffle(filtered_data)

# Calculate the split index for 80/20 split
split_index = int(len(filtered_data) * 0.8)

# Split the data into training and validation sets
train_data = filtered_data[:split_index]
valid_data = filtered_data[split_index:]

# Update the 'split' field in each entry
for entry in train_data:
    entry['split'] = 'train'

for entry in valid_data:
    entry['split'] = 'valid'

# Write the training data to 'human_scanpath_train_split.json'
with open(train_file_path, 'w') as train_file:
    json.dump(train_data, train_file, indent=4)

# Write the validation data to 'human_scanpath_valid_split.json'
with open(valid_file_path, 'w') as valid_file:
    json.dump(valid_data, valid_file, indent=4)

print("Data has been filtered, split, and written to the respective files.")
print(f"Total entries after filtering: {len(filtered_data)} and {len(data) - len(filtered_data)} entries were removed.")
print(f"Training entries: {len(train_data)}")
print(f"Validation entries: {len(valid_data)}")


Data has been filtered, split, and written to the respective files.
Total entries after filtering: 257 and 10 entries were removed.
Training entries: 205
Validation entries: 52


In [5]:
import json
import random
import os

# Specify the path to your JSON file
json_file_path = 'fixation_data.json'

# Output file paths
train_file_path = 'stroop_human_scanpath_train_split.json'
valid_file_path = 'stroop_human_scanpath_valid_split.json'

# Check if the input file exists
if not os.path.exists(json_file_path):
    print(f"File not found: {json_file_path}")
    exit(1)

# Load the JSON data
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Add 'featureA', 'featureB', and 'featureC' to each entry
for entry in data:
    # length = entry.get('length', 0)
    length = 3
    # Ensure length is an integer and greater than 0
    if isinstance(length, int) and length > 0:
        # Generate random values between 0 and 1
        entry['eeg_data'] = [random.random() for _ in range(length)]
    else:
        # Handle entries without valid 'length'
        entry['eeg_data'] = []

# Thresholds
x_threshold = 512
y_threshold = 320

# Filter entries that have no X[i] > 512 and no Y[i] > 320
filtered_data = []
for idx, entry in enumerate(data):
    X_values = entry.get('X', [])
    Y_values = entry.get('Y', [])

    # Check if any X[i] > 512 or Y[i] > 320
    x_exceeds = any(x > x_threshold for x in X_values)
    y_exceeds = any(y > y_threshold for y in Y_values)

    if not x_exceeds and not y_exceeds:
        filtered_data.append(entry)

# Shuffle the filtered data
random.shuffle(filtered_data)

# Calculate the split index for 80/20 split
split_index = int(len(filtered_data) * 0.8)

# Split the data into training and validation sets
train_data = filtered_data[:split_index]
valid_data = filtered_data[split_index:]

# Update the 'split' field in each entry
for entry in train_data:
    entry['split'] = 'train'

for entry in valid_data:
    entry['split'] = 'valid'

# Write the training data to 'human_scanpath_train_split.json'
with open(train_file_path, 'w') as train_file:
    json.dump(train_data, train_file, indent=4)

# Write the validation data to 'human_scanpath_valid_split.json'
with open(valid_file_path, 'w') as valid_file:
    json.dump(valid_data, valid_file, indent=4)

print("Data has been filtered, split, and written to the respective files.")
print(f"Total entries after filtering: {len(filtered_data)} and {len(data) - len(filtered_data)} entries were removed.")
print(f"Training entries: {len(train_data)}")
print(f"Validation entries: {len(valid_data)}")


Data has been filtered, split, and written to the respective files.
Total entries after filtering: 142 and 56 entries were removed.
Training entries: 113
Validation entries: 29


In [ ]:
import json
import random
import os

# Specify the path to your JSON file
json_file_path = 'fixation_data.json'

# Output file paths
train_file_path = 'stroop_human_scanpath_train_split.json'
valid_file_path = 'stroop_human_scanpath_valid_split.json'

# Check if the input file exists
if not os.path.exists(json_file_path):
    print(f"File not found: {json_file_path}")
    exit(1)

# Load the JSON data
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Thresholds
x_threshold = 512
y_threshold = 320

# Filter entries that have no X[i] > 512 and no Y[i] > 320
filtered_data = []
removed_entries = []


# Create the list of slides ['Slide1.jpg', 'Slide3.jpg', ..., 'Slide59.jpg']
target_slides = [f'Slide{2*i-1}.jpg' for i in range(1, 20)]
print("target_slides: " + str(target_slides))

# Create a dictionary to track which slides each subject has
subjects_slides = {subject: set() for subject in range(1, 12)}

# Populate the dictionary with available slides for each subject
for record in data:
    subject = record['subject']
    slide_name = record['name']
    if subject in subjects_slides:
        subjects_slides[subject].add(slide_name)

# Now check for each subject which slides are missing
for subject in range(1, 12):
    available_slides = subjects_slides[subject]
    missing_slides = [slide for slide in target_slides if slide not in available_slides]
    
    if missing_slides:
        print(f"Subject {subject} is missing the following slides: {', '.join(missing_slides)}")
    else:
        print(f"Subject {subject} has all the slides.")




target_slides: ['Slide1.jpg', 'Slide3.jpg', 'Slide5.jpg', 'Slide7.jpg', 'Slide9.jpg', 'Slide11.jpg', 'Slide13.jpg', 'Slide15.jpg', 'Slide17.jpg', 'Slide19.jpg', 'Slide21.jpg', 'Slide23.jpg', 'Slide25.jpg', 'Slide27.jpg', 'Slide29.jpg', 'Slide31.jpg', 'Slide33.jpg', 'Slide35.jpg', 'Slide37.jpg']
Subject 1 has all the slides.
Subject 2 has all the slides.
Subject 3 is missing the following slides: Slide19.jpg, Slide21.jpg
Subject 4 has all the slides.
Subject 5 has all the slides.
Subject 6 has all the slides.
Subject 7 is missing the following slides: Slide1.jpg, Slide3.jpg, Slide5.jpg, Slide7.jpg, Slide9.jpg, Slide11.jpg, Slide13.jpg, Slide15.jpg, Slide17.jpg, Slide19.jpg, Slide21.jpg, Slide23.jpg, Slide25.jpg, Slide27.jpg, Slide29.jpg, Slide31.jpg, Slide33.jpg, Slide35.jpg, Slide37.jpg
Subject 8 has all the slides.
Subject 9 has all the slides.
Subject 10 has all the slides.
Subject 11 has all the slides.


# Build Dataset

In [1]:
import pandas as pd
import json

def read_csv(file_path):
    # Read CSV file with comma delimiter
    df = pd.read_csv(file_path, delimiter=',')
    # Clean column names to remove leading/trailing whitespaces and standardize them
    df.columns = df.columns.str.strip().str.upper()
    return df

def extract_rows(df, start, end, time_column):
    # Convert time column to numeric
    df['timestamp'] = pd.to_numeric(df[time_column], errors='coerce')
    # Drop rows with NaN values in 'timestamp' due to conversion issues
    df.dropna(subset=['timestamp'], inplace=True)
    # Extract rows within the given time range
    mask = (df['timestamp'] >= start) & (df['timestamp'] <= end)
    return df[mask]

def is_within_bbox(x, y, bbox):
    x_min, y_min, width, height = bbox
    x_max = x_min + width
    y_max = y_min + height

    # Ensure that x_min <= x_max and y_min <= y_max
    x_min, x_max = sorted([x_min, x_max])
    y_min, y_max = sorted([y_min, y_max])

    return x_min <= x <= x_max and y_min <= y <= y_max

def generate_json_entry(name, subject, task, condition, bbox, filtered_df, cx_column, cy_column, duration_column, start_time_in_sec, end_time_in_sec, time_range_in_minutes):
    # X_real = filtered_df[cx_column].tolist()
    # Y_real = filtered_df[cy_column].tolist()
    # Hey chatgpt, Here I want if each if any of X or Y is less than zero put is as zero. For example 
    # if X is -2 put it as 0 and if Y is -3 put it as 0.
    X = [(min(max(0, x * 512),511)) for x in filtered_df[cx_column].tolist()]
    Y = [(min(max(0, y * 320),319)) for y in filtered_df[cy_column].tolist()]
    T = filtered_df[duration_column].tolist()
    length = len(X)
    correct = any(is_within_bbox(X[i], Y[i], bbox) for i in range(length))
    
    return {
        "name": name,
        "subject": subject,
        # "time_range_in_seconds": [start_time_in_sec, end_time_in_sec],
        # "time_range_in_minutes": time_range_in_minutes,
        "task": task,
        "condition": condition,
        "bbox": bbox,
        # "X_real": X_real,
        # "Y_real": Y_real, 
        "X": X,
        "Y": Y,
        "T": T,
        "length": length,
        "correct": int(correct),
        "split": "train"
    }

def convert_time_to_seconds(time_str):
    # Convert time format (HH:MM:SS:MS) to seconds
    hours, minutes, seconds, milliseconds = map(int, time_str.split(':'))
    return hours * 3600 + minutes * 60 + seconds + milliseconds / 1000

def extract_fixations(file_paths, entries, width=512, height=320):
    results = []
    BPOGX_column = 'BPOGX'
    BPOGY_column = 'BPOGY'
    time_column = 'FPOGD'

    for entry in entries:
        subject = entry['subject']
        file_path = file_paths.get(subject)
        if not file_path:
            continue

        df = read_csv(file_path)

        if BPOGX_column not in df.columns or BPOGY_column not in df.columns or time_column not in df.columns:
            raise KeyError("Columns 'BPOGX', 'BPOGY', or 'FPOGD' not found in CSV file.")

        name = entry['name']
        update_dict = {
            "Slide1.jpg": "congruent",
            "Slide3.jpg": "incongruent",
            "Slide5.jpg": "congruent", 
            "Slide7.jpg": "congruent",
            "Slide9.jpg": "incongruent",
            "Slide11.jpg": "congruent",
            "Slide13.jpg": "incongruent",
            "Slide15.jpg": "incongruent",
            "Slide17.jpg": "incongruent",
            "Slide19.jpg": "incongruent",
            "Slide21.jpg": "congruent",
            "Slide23.jpg": "incongruent",
            "Slide25.jpg": "congruent",
            "Slide27.jpg": "incongruent",
            "Slide29.jpg": "congruent",
            "Slide31.jpg": "incongruent",
            "Slide33.jpg": "congruent",
            "Slide35.jpg": "incongruent",
            "Slide37.jpg": "incongruent",
            "Slide39.jpg": "congruent",
        }
        task = entry['task']
        entry["task"] = update_dict[name]
        condition = entry['condition']
        bbox_indicator = entry['bbox_indicator']
        bbox = [181, 10, 149, 33] if bbox_indicator == 1 else [181, 276, 149, 34]
        start, end = entry['time_range']

        # Convert time range to seconds
        start_time = convert_time_to_seconds(start)
        end_time = convert_time_to_seconds(end)

        filtered_df = extract_rows(df, start_time, end_time, 'FPOGS')
        json_entry = generate_json_entry(name, subject, task, condition, bbox, filtered_df, BPOGX_column, BPOGY_column, time_column, start_time, end_time, entry['time_range'])
        if not filtered_df.empty: # Subject 3 Slide19 and Slide21 have empty fixations
            results.append(json_entry)

    # Limit the size of the JSON by rounding values to reduce length
    for entry in results:
        entry['X'] = [round(x, 2) for x in entry['X']]
        entry['Y'] = [round(y, 2) for y in entry['Y']]
        entry['T'] = [round(t*1000) for t in entry['T']]
    
    return results

# List of file paths for each subject
file_paths = {
    1: '/home/ali/Repos/Scanpath_Prediction/files/Raw_Gaze_Data/P1/GAZE/result/User 0_fixations.csv',
    2: '/home/ali/Repos/Scanpath_Prediction/files/Raw_Gaze_Data/P2/GAZE/result/User 2_fixations.csv',
    3: '/home/ali/Repos/Scanpath_Prediction/files/Raw_Gaze_Data/P3/GAZE/result/User 1_fixations.csv',
    4: '/home/ali/Repos/Scanpath_Prediction/files/Raw_Gaze_Data/P4/GAZE/result/User 0_fixations.csv',
    5: '/home/ali/Repos/Scanpath_Prediction/files/Raw_Gaze_Data/P5/GAZE/result/User 0_fixations.csv',
    6: '/home/ali/Repos/Scanpath_Prediction/files/Raw_Gaze_Data/P6/Raw Data/GAZE/result/User 0_fixations.csv',
    8: '/home/ali/Repos/Scanpath_Prediction/files/Raw_Gaze_Data/P8/GAZE/result/User 0_fixations.csv',
    9: '/home/ali/Repos/Scanpath_Prediction/files/Raw_Gaze_Data/P9/GAZE/result/User 1_fixations.csv',
    10: '/home/ali/Repos/Scanpath_Prediction/files/Raw_Gaze_Data/P10/GAZE/result/User 0_fixations.csv',
    11: '/home/ali/Repos/Scanpath_Prediction/files/Raw_Gaze_Data/P11/GAZE/result/User 0_fixations.csv',
}

# Load list of entries for fixation extraction from JSON file
with open('entries_corrected.json', 'r') as entries_file:
    entries = json.load(entries_file)

# Extract and scale fixation data
fixation_data = extract_fixations(file_paths, entries)

# Convert result to JSON
with open('fixation_data.json', 'w') as json_file:
    json.dump(fixation_data, json_file, indent=4)


In [2]:
# Uses only for entring data to make all of the times to zero.
# import json

# # Read the JSON file
# with open('entries.json', 'r') as file:
#     data = json.load(file)

# # Modify the time_range for each entry
# for entry in data:
#     entry['time_range'] = ["00:00:00:00", "00:00:00:00"]

# # Write to the new JSON file with the time_range in a single line
# with open('entries_corrected.json', 'w') as file:
#     json.dump(data, file, indent=4)

In [3]:
import json

# Function to update bbox_indicator based on the given dictionary
def update_bbox_indicator(file_path, update_dict):
    # Load the JSON data from file
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Loop through each entry in the JSON data
    for entry in data:
        name = entry.get("name")
        # Check if the name is in the update dictionary
        if name in update_dict:
            # Update bbox_indicator with the value from the dictionary
            entry["task"] = update_dict[name]
    
    # Write the updated data back to the JSON file
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)
    print(f"Updated entries saved to {file_path}")

# Example usage
file_path = 'fixation_data.json'
update_dict = {
    "Slide1.jpg": "congruent",
    "Slide3.jpg": "incongruent",
    "Slide5.jpg": "congruent", 
    "Slide7.jpg": "congruent",
    "Slide9.jpg": "incongruent",
    "Slide11.jpg": "congruent",
    "Slide13.jpg": "incongruent",
    "Slide15.jpg": "incongruent",
    "Slide17.jpg": "incongruent",
    "Slide19.jpg": "incongruent",
    "Slide21.jpg": "congruent",
    "Slide23.jpg": "incongruent",
    "Slide25.jpg": "congruent",
    "Slide27.jpg": "incongruent",
    "Slide29.jpg": "congruent",
    "Slide31.jpg": "incongruent",
    "Slide33.jpg": "congruent",
    "Slide35.jpg": "incongruent",
    "Slide37.jpg": "incongruent",
    "Slide39.jpg": "congruent",
}

update_bbox_indicator(file_path, update_dict)


Updated entries saved to fixation_data.json


In [4]:
# Set Correct EEGs of the Stroop to the fixation_data.json
import json
import pandas as pd

# Load the JSON data
with open("fixation_data.json", "r") as json_file:
    fixation_data = json.load(json_file)

# Load the Excel data
excel_data = pd.read_excel("files/MostImpEEG_Stroop.xlsx")

# Print column names to inspect them
print("Column names in Excel file:", excel_data.columns)

# Assuming the columns are not labeled B, C, D, and E, replace them with actual column names
# Iterate over each JSON record and update based on Excel data
excel_index = 0  # Start from the first row in the Excel file
skipped_ids = {48, 49}  # Corresponding IDs for subject 3, slides 19 and 21

for record in fixation_data:
    subject = record["subject"]
    slide_number = int(record["name"].replace("Slide", "").replace(".jpg", ""))
    
    # Calculate the id based on subject and slide
    record_id = (subject - 1) * 20 + (slide_number // 2) + 1
    

    
    # Update 'eeg_data' field from F1, F2, and F3 in the Excel file
    record["eeg_data"] = [
        excel_data.iloc[excel_index][excel_data.columns[1]],  # F1
        excel_data.iloc[excel_index][excel_data.columns[2]],  # F2
        excel_data.iloc[excel_index][excel_data.columns[3]],  # F3
    ]


    # I have disabled this currently. I have set in another place
    # Update 'correct' field based on label
    # label = excel_data.iloc[excel_index][excel_data.columns[4]]
    # record["task"] = "congruent" if label == "C" else "incongruent"
    # print(f"subject:{subject} slide_number:{slide_number} record_id:{record_id}" + 
    #       f"excel_index:{excel_index} eeg:{record["eeg_data"]}")
    
    # Skip specific slides for subject 3 (slide 19 and slide 21)
    if subject == 3 and slide_number == 17:
        excel_index += 3
        continue
    # Increment Excel index to move to the next row
    else:
        excel_index += 1

# Save the updated JSON data
with open("fixation_data.json", "w") as json_file:
    json.dump(fixation_data, json_file, indent=4)

print("JSON file updated successfully!")

Column names in Excel file: Index(['id', 'F1', 'F2', 'F3', 'labels'], dtype='object')
JSON file updated successfully!


In [5]:
import json
import numpy as np

# Load the JSON file
with open('fixation_data.json', 'r') as f:
    data = json.load(f)

# Create a dictionary to store the data in the desired format
bbox_annos = {}

# Extract the necessary fields and populate the dictionary
for entry in data:
    name = entry['name']
    task = entry['task']
    bbox = entry['bbox']
    
    # Combine name and task to create a unique key for each entry
    key = f"{task}_{name}"
    bbox_annos[key] = bbox

# Save the dictionary as a .npy file
np.save('files/Stroop_DataSet/bbox_annos.npy', bbox_annos)

# To load and verify the .npy file
loaded_bbox_annos = np.load('files/Stroop_DataSet/bbox_annos.npy', allow_pickle=True).item()
print(loaded_bbox_annos)

{'congruent_Slide1.jpg': [181, 10, 149, 33], 'incongruent_Slide3.jpg': [181, 276, 149, 34], 'congruent_Slide5.jpg': [181, 10, 149, 33], 'congruent_Slide7.jpg': [181, 10, 149, 33], 'incongruent_Slide9.jpg': [181, 276, 149, 34], 'congruent_Slide11.jpg': [181, 10, 149, 33], 'incongruent_Slide13.jpg': [181, 276, 149, 34], 'incongruent_Slide15.jpg': [181, 276, 149, 34], 'incongruent_Slide17.jpg': [181, 276, 149, 34], 'incongruent_Slide19.jpg': [181, 276, 149, 34], 'congruent_Slide21.jpg': [181, 10, 149, 33], 'incongruent_Slide23.jpg': [181, 276, 149, 34], 'congruent_Slide25.jpg': [181, 10, 149, 33], 'incongruent_Slide27.jpg': [181, 276, 149, 34], 'congruent_Slide29.jpg': [181, 10, 149, 33], 'incongruent_Slide31.jpg': [181, 276, 149, 34], 'congruent_Slide33.jpg': [181, 10, 149, 33], 'incongruent_Slide35.jpg': [181, 276, 149, 34], 'incongruent_Slide37.jpg': [181, 276, 149, 34], 'congruent_Slide39.jpg': [181, 10, 149, 33]}


# Train/Valid Split

In [6]:
import json
import random
import os

# Specify the path to your JSON file
json_file_path = 'fixation_data.json'

# Output file paths
train_file_path = 'files/Stroop_DataSet/stroop_human_scanpath_train_split.json'
valid_file_path = 'files/Stroop_DataSet/stroop_human_scanpath_valid_split.json'

# Check if the input file exists
if not os.path.exists(json_file_path):
    print(f"File not found: {json_file_path}")
    exit(1)

# Load the JSON data
with open(json_file_path, 'r') as file:
    data = json.load(file)

random.seed(42)
# Shuffle the filtered data
random.shuffle(data)

# Calculate the split index for 80/20 split
split_index = int(len(data) * 0.8)

# Split the data into training and validation sets
train_data = data[:split_index]
valid_data = data[split_index:]

# Update the 'split' field in each entry
for entry in train_data:
    entry['split'] = 'train'

for entry in valid_data:
    entry['split'] = 'valid'

# Write the training data to 'human_scanpath_train_split.json'
with open(train_file_path, 'w') as train_file:
    json.dump(train_data, train_file, indent=4)

# Write the validation data to 'human_scanpath_valid_split.json'
with open(valid_file_path, 'w') as valid_file:
    json.dump(valid_data, valid_file, indent=4)

print("Data has been filtered, split, and written to the respective files.")
print(f"Total entries after filtering: {len(data)}")
print(f"Training entries: {len(train_data)}")
print(f"Validation entries: {len(valid_data)}")

Data has been filtered, split, and written to the respective files.
Total entries after filtering: 198
Training entries: 158
Validation entries: 40


# Resize Images

In [7]:
import os
from PIL import Image

def resize_images(source_dir, dest_dir, size=(512, 320)):
    # Create destination directory if it doesn't exist
    os.makedirs(dest_dir, exist_ok=True)
    
    # Iterate over all files in the source directory
    for filename in os.listdir(source_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):  # Check for image files
            file_path = os.path.join(source_dir, filename)
            with Image.open(file_path) as img:
                # Resize image
                img = img.resize(size)
                # Save resized image to destination directory
                img.save(os.path.join(dest_dir, filename))
                print(f"Saved resized image: {filename} to {dest_dir}")

# Do the resizing for the Stroop dataset (Congruent and Incongruent)
source_directory = 'files/Stroop_DataSet/Stroop/congruent'
destination_directory = 'files/Stroop_DataSet/Stroop 512x320/congruent'
resize_images(source_directory, destination_directory)
source_directory = 'files/Stroop_DataSet/Stroop/incongruent'
destination_directory = 'files/Stroop_DataSet/Stroop 512x320/incongruent'
resize_images(source_directory, destination_directory)

Saved resized image: Slide1.png to files/Stroop_DataSet/Stroop 512x320/congruent
Saved resized image: Slide33.png to files/Stroop_DataSet/Stroop 512x320/congruent
Saved resized image: Slide5.png to files/Stroop_DataSet/Stroop 512x320/congruent
Saved resized image: Slide25.png to files/Stroop_DataSet/Stroop 512x320/congruent
Saved resized image: Slide7.png to files/Stroop_DataSet/Stroop 512x320/congruent
Saved resized image: Slide29.png to files/Stroop_DataSet/Stroop 512x320/congruent
Saved resized image: Slide21.png to files/Stroop_DataSet/Stroop 512x320/congruent
Saved resized image: Slide39.png to files/Stroop_DataSet/Stroop 512x320/congruent
Saved resized image: Slide11.png to files/Stroop_DataSet/Stroop 512x320/congruent
Saved resized image: Slide17.png to files/Stroop_DataSet/Stroop 512x320/incongruent
Saved resized image: Slide9.png to files/Stroop_DataSet/Stroop 512x320/incongruent
Saved resized image: Slide35.png to files/Stroop_DataSet/Stroop 512x320/incongruent
Saved resized 

# Compute DCBs

In [8]:
import torch
import torch.nn.functional as F
import numpy as np
import os

from PIL import Image, ImageFilter
from detectron2.config import get_cfg
from detectron2.modeling import build_backbone
from detectron2.engine import DefaultPredictor

device = torch.device("cuda")


def pred2feat(seg, info):
    seg = seg.cpu()
    feat = torch.zeros([80 + 54, 320, 512])
    for pred in info:
        mask = (seg == pred['id']).float()
        if pred['isthing']:
            feat[pred['category_id'], :, :] = mask * pred['score']
        else:
            feat[pred['category_id'] + 80, :, :] = mask
    return F.interpolate(feat.unsqueeze(0), size=[20, 32]).squeeze(0)


def get_DCBs(img_path, predictor, radius=1):
    high = Image.open(img_path).convert('RGB').resize((512, 320))
    low = high.filter(ImageFilter.GaussianBlur(radius=radius))
    high_panoptic_seg, high_segments_info = predictor(
        np.array(high))["panoptic_seg"]
    low_panoptic_seg, low_segments_info = predictor(
        np.array(low))["panoptic_seg"]
    high_feat = pred2feat(high_panoptic_seg, high_segments_info)
    low_feat = pred2feat(low_panoptic_seg, low_segments_info)
    return high_feat, low_feat


if __name__ == '__main__':
    # Load pretrained panoptic_fpn
    cfg = get_cfg()
    cfg.merge_from_file(
        './detectron2/configs/COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml'
    )
    cfg.MODEL.WEIGHTS = 'detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_50_3x/139514569/model_final_c10459.pkl'
    model = build_backbone(cfg).to(device)
    model.eval()

    predictor = DefaultPredictor(cfg)

    # Specify directories
    img_dir = 'files/Stroop_DataSet/Stroop 512x320'
    hr_dir = 'files/Stroop_DataSet/DCBs/HR'
    lr_dir = 'files/Stroop_DataSet/DCBs/LR'
    
    # Ensure output directories exist
    os.makedirs(hr_dir, exist_ok=True)
    os.makedirs(lr_dir, exist_ok=True)

    # Process each image file in the directory recursively
    for root, dirs, files in os.walk(img_dir):
        for img_filename in files:
            # Check if the file is an image
            if img_filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, img_filename)

                # Get high and low resolution features
                high_feat, low_feat = get_DCBs(img_path, predictor)
                
                # Create relative path and ensure the output directories exist
                relative_path = os.path.relpath(root, img_dir)
                hr_output_dir = os.path.join(hr_dir, relative_path)
                lr_output_dir = os.path.join(lr_dir, relative_path)
                
                os.makedirs(hr_output_dir, exist_ok=True)
                os.makedirs(lr_output_dir, exist_ok=True)

                # Save features to HR and LR directories
                image_filename = f'{img_filename.replace(".png", "").replace(" ", "")}.pth.tar'
                torch.save(high_feat, os.path.join(hr_output_dir, image_filename))
                torch.save(low_feat, os.path.join(lr_output_dir, image_filename))

                print(f"Processed and saved features for {img_filename}")


/home/ali/Repos/Scanpath_Prediction/.venv/lib/python3.12/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Processed and saved features for Slide 11.png
Processed and saved features for Slide1.png
Processed and saved features for Slide 39.png
Processed and saved features for Slide 21.png
Processed and saved features for Slide 33.png
Processed and saved features for Slide 5.png
Processed and saved features for Slide33.png
Processed and saved features for Slide5.png
Processed and saved features for Slide 1.png
Processed and saved features for Slide25.png
Processed and saved features for Slide 7.png
Processed and saved features for Slide7.png
Processed and saved features for Slide 25.png
Processed and saved features for Slide 29.png
Processed and saved features for Slide29.png
Processed and saved features for Slide21.png
Processed and saved features for Slide39.png
Processed and saved features for Slide11.png
Processed and saved features for Slide17.png
Processed and saved features for Slide 27.png
Processed and saved features for Slide9.png
Processed and saved features for Slide35.png
Process

# SWAP  Xs and Ys

In [9]:
# import json

# # Load the data from the fixation.json file
# with open('files/Stroop_DataSet/stroop_human_scanpath_train_split.json', 'r') as file:
#     data = json.load(file)

# # Swap X and Y values for all entries
# for entry in data:
#     entry['X'], entry['Y'] = entry['Y'], entry['X']

# # Save the modified data back to a new file
# with open('files/Stroop_DataSet/stroop_human_scanpath_train_split.json', 'w') as file:
#     json.dump(data, file, indent=4)

#     # Load the data from the fixation.json file
# with open('files/Stroop_DataSet/stroop_human_scanpath_valid_split.json', 'r') as file:
#     data = json.load(file)

# # Swap X and Y values for all entries
# for entry in data:
#     entry['X'], entry['Y'] = entry['Y'], entry['X']

# # Save the modified data back to a new file
# with open('files/Stroop_DataSet/stroop_human_scanpath_valid_split.json', 'w') as file:
#     json.dump(data, file, indent=4)